In [1]:
!pip install gradio >installs.txt
!pip install langchain >installs.txt
!pip install pypdf >installs.txt
!pip install --quiet langchain_experimental python-dotenv langchain_openai langchainhub >installs.txt
!pip install chromadb >installs.txt



In [5]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains import RetrievalQA
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.tools.retriever import create_retriever_tool
import gradio as gr
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [15]:
def process_pdf(pdf_path):
    """
    Processes a PDF document by loading, splitting, and embedding its text,
    then storing the embeddings in a Chroma vector store.

    Args:
        pdf_path (str): The path to the PDF document to be processed.

    Returns:
        str: Confirmation message that the file is uploaded successfully.
    """
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    text_splitter = SemanticChunker(OpenAIEmbeddings())
    texts = text_splitter.split_documents(pages)
    vectorstore = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings(), persist_directory="./chroma_db")
    vectorstore.persist()
    return "Your File is Uploaded Successfully!"
def rag_inference(question):
    """
    Performs retrieval-augmented generation (RAG) to answer a question using
    the document embeddings stored in a Chroma vector store.

    Args:
        question (str): The question to be answered.

    Returns:
        str: The generated answer to the question.
    """
    vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_kwargs={"k": vectorstore._collection.count()})
    llm = ChatOpenAI(api_key=api_key, model_name="gpt-4-turbo", temperature=0)
    prompt = hub.pull("hwchase17/react")
    tool_search = create_retriever_tool(
        retriever=retriever,
        name="search_document",
        description="Searches and returns excerpts from the document.",
    )
    react_agent = create_react_agent(llm, [tool_search], prompt)
    agent_executor = AgentExecutor(
        agent=react_agent,
        tools=[tool_search],
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=10
    )
    result = agent_executor.invoke({"input": question})
    return result['output']


In [16]:
import gradio as gr
def upload_and_process(pdf_file):
    return process_pdf(pdf_file.name)

def ask_question(question):
    return rag_inference(question)

with gr.Blocks() as demo:
    gr.Markdown("## TASK-1 Mini RAg Bot")

    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", type="filepath", file_count="single")
        upload_button = gr.Button("Upload and Process")
        output_text = gr.Textbox()
        upload_button.click(upload_and_process, inputs=pdf_input, outputs=output_text)
    
    with gr.Row():
        question_input = gr.Textbox(label="Ask a question about the PDF")
        ask_button = gr.Button("Ask")
        answer_output = gr.Textbox(label="Answer", interactive=False)
        ask_button.click(ask_question, inputs=question_input, outputs=answer_output)

demo.launch()




Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
I need to understand the content of the document to answer the question about its topic.

Action: search_document
Action Input: "document summary"Categories:  Language Modelling  
Links:  Paper, Code  
 
Title:  VideoLLaMA 2: Advancing Spatial -Temporal Modeling and Audio Understanding in 
Video -LLMs  
Authors:  damo -nlp-sg/videollama2  
Date:  11 Jun 2024  
Description:  In this paper, we present the VideoLLaMA 2, a set of Video Large Language 
Models (Video -LLMs) designed to enhance spatial -temporal modeling and audio understanding 
in video and audio -oriented tasks. Stats:  318, 1.50 stars / hour  
Categories:  Multiple -choice, Question Answering +3  
Links:  Paper, Code

Categories:  Language Modelling  
Links:  Paper, Code  
 
Title:  VideoLLaMA 2: Advancing Spatial -Temporal Modeling and Audio Understanding in 
Video -LLMs  
Authors:  damo -nlp-sg/videollama2  
Date:  11 Jun 2024  
Description:  In this paper, we present the VideoLLaM